In [3]:
library("tidyverse")
library("data.table")

# Fish numbers

In [6]:
df <- fread("/nfs/research/birney/users/saul/nextflow/medaka_behaviour_pilot/hmm/time_step0.08_n_states15_hmm.csv.gz")

In [7]:
ids <- df[, unique(id)]

In [8]:
pairs <- str_remove(ids, "_q.") |> str_remove("_(L|R)") |> str_remove("^[0-9]*_[0-9]*_") |> table()

In [9]:
pairs


  icab_hdr_no_ref  icab_hdr_no_test   icab_hdr_of_ref  icab_hdr_of_test 
               60                60                60                60 
  icab_hni_no_ref  icab_hni_no_test   icab_hni_of_ref  icab_hni_of_test 
               76                76                76                76 
  icab_ho5_no_ref  icab_ho5_no_test   icab_ho5_of_ref  icab_ho5_of_test 
               56                56                56                56 
 icab_icab_no_ref icab_icab_no_test  icab_icab_of_ref icab_icab_of_test 
               68                68                68                68 
 icab_kaga_no_ref icab_kaga_no_test  icab_kaga_of_ref icab_kaga_of_test 
               47                47                47                47 

In [10]:
pairs[grepl("of_ref", names(pairs))]


 icab_hdr_of_ref  icab_hni_of_ref  icab_ho5_of_ref icab_icab_of_ref 
              60               76               56               68 
icab_kaga_of_ref 
              47 

In [24]:
pairs[grepl("no_ref", names(pairs))]


 icab_hdr_no_ref  icab_hni_no_ref  icab_ho5_no_ref icab_icab_no_ref 
              60               76               56               68 
icab_kaga_no_ref 
              47 

# Tracking accuracy

In [2]:
f_list <- list.files("/lts/research/birney/users/saul/nextflow/medaka_behaviour_pilot/trajectories_csv/", full.names = TRUE)

In [3]:
df <- lapply(
    f_list,
    function(f){
        fread(f)[, id := str_remove(basename(f), "_traj_(with_identities)?.csv.gz")]
    }
) |> rbindlist(use.names = TRUE, fill = TRUE)

In [4]:
df <- df[, .(frame = 1:.N, ref_x, ref_y, test_x, test_y), by = id]

In [5]:
df <- melt(df, id.vars = c("id", "frame"))

In [6]:
df[, fish := str_remove(variable, "_(x|y)")]
df[, variable := str_remove(variable, "(ref|test)_")]

In [7]:
df <- dcast(df, id + frame + fish ~ variable, value.var = "value")

In [8]:
summary <- df[, .(tracked_fraction = mean(!is.na(x) & !is.na(y))), by = c("id", "fish")]
remove(df)

In [9]:
summary[, min(tracked_fraction)]

[1] 0.7800556

In [11]:
summary[, mean(tracked_fraction >= 0.99)]

[1] 0.8802932